# FINAL PROJECT - Introduction to Data Science

# Data Collecting

---

### Import Libraries

In [ ]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By

import re
import pandas as pd

In [ ]:
BASE_URL = "https://www.basketball-reference.com/"
# https://www.basketball-reference.com/leagues/NBA_2024.html

---
### Eastern/Western Conference Standings

In [57]:
def get_conf_std(year_interval: str='2015:2024', conf: str='E'):
    all_data = []
    start_year, end_year = year_interval.split(':')
    for year in range(int(start_year), int(end_year) + 1):
        # Path to Edge WebDriver
        service = Service('C:\Program Files\WebDriver\msedgedriver.exe')

        # Initialize ME browser
        driver = webdriver.Edge(service=service)
        # Construct the URL for each year
        url = f"{BASE_URL}leagues/NBA_{year}.html"
        driver.get(url)
        
        # Locate the Conference standings
        try: 
            conference = driver.find_element(By.ID, f'all_confs_standings_{conf}')
        except Exception:
            try:
                conference = driver.find_element(By.ID, f'all_divs_standings_{conf}')
            except Exception:
                driver.quit()
                continue
        
        # Extract column names
        header = conference.find_element(By.TAG_NAME, 'thead')
        cols = ['Eastern Conference'] + header.text.split(' ')[2:]
        
        # Extract the rows with data
        rows = conference.find_elements(By.TAG_NAME, 'tr')
        
        # Prepare lists to store data
        team_names, Wins, Losses, wlpcts, gbs, pts, opp_pts, srss = [], [], [], [], [], [], [], []
        
        # Extract data from each row
        for row in rows[1:]:  # Skipping the header row
            try:
                team_name = row.find_element(By.CSS_SELECTOR, "th[data-stat='team_name']")
                wins = row.find_element(By.CSS_SELECTOR, "td[data-stat='wins']")
                losses = row.find_element(By.CSS_SELECTOR, "td[data-stat='losses']")
                win_loss_pct = row.find_element(By.CSS_SELECTOR, "td[data-stat='win_loss_pct']")
                gb = row.find_element(By.CSS_SELECTOR, "td[data-stat='gb']")
                pts_per_g = row.find_element(By.CSS_SELECTOR, "td[data-stat='pts_per_g']")
                opp_pts_per_g = row.find_element(By.CSS_SELECTOR, "td[data-stat='opp_pts_per_g']")
                srs = row.find_element(By.CSS_SELECTOR, "td[data-stat='srs']")
                
                team_names.append(team_name.text.strip('*'))
                Wins.append(int(wins.text))
                Losses.append(int(losses.text))
                wlpcts.append(float(win_loss_pct.text))
                gbs.append(float(gb.text) if gb.text != '—' else 0.0)
                pts.append(float(pts_per_g.text))
                opp_pts.append(float(opp_pts_per_g.text))
                srss.append(float(srs.text))
            
            except Exception:
                continue
        
        # Create a DataFrame for the current year
        data = list(zip(team_names, Wins, Losses, wlpcts, gbs, pts, opp_pts, srss))
        df_year = pd.DataFrame(data, columns=cols)
        df_year['Year'] = year  # Add a column for the year
        
        # Append the year data to the all_data list
        all_data.append(df_year)

    # Concatenate all the data frames
    final_df = pd.concat(all_data, axis=0, ignore_index=True)
    driver.quit()
    
    return final_df

In [ ]:
E_df = get_conf_std('1995:2024', 'E')
E_df

In [ ]:
E_df.to_csv('../Data/eastern_conference_standings.csv', sep=',', encoding='utf-8', index=False)

In [ ]:
W_df = get_conf_std('1995:2024', 'W')
W_df

In [ ]:
W_df.to_csv('../Data/western_conference_standings.csv', sep=',', encoding='utf-8', index=False)

---
### Historical Players

In [ ]:
def get_historical_players(year_interval: str='1995:2024'):
    all_data = []
    start_year, end_year = year_interval.split(':')
    for year in range(int(start_year), int(end_year) + 1):
        service = Service('C:\Program Files\WebDriver\msedgedriver.exe')
        # Initialize ME browser
        driver = webdriver.Edge(service=service)
        # Construct the URL for each year
        url = f"{BASE_URL}leagues/NBA_{year}_ratings.html"
        driver.get(url)

        try: 
            team_ratings = driver.find_element(By.ID, 'ratings')
        except Exception as e:
            print(f'Error: {e}')
            driver.quit()
            continue

        # Extract column names
        cols = ['Rank', 'Team', 'Conf', 'Div', 'Wins', 'Losses', 'Win-Loss Percentage', 'Margin of Victory', 'Offensive Rating', 'Defensive Rating', 'Net Rating', 'Adjusted Margin of Victory', 'Adjusted Offensive Victory', 'Adjusted Defensive Rating', 'Adjusted Net Rating']

        rows = team_ratings.find_element(By.TAG_NAME, 'tbody')
        data_rows = rows.find_elements(By.TAG_NAME, 'tr')
        rks, teams, confs, divs, wins, losses, wlps, movs, orts, drts, nrts, movas, ortas, drtas, nrtas  = [], [], [], [], [], [], [], [], [], [], [], [], [], [], []
        # Extract data
        for row in data_rows:
            rank = row.find_element(By.TAG_NAME, 'th').text
            try:
                rank = int(rank)
            except:
                continue
            rks.append(rank)
            stats = row.find_elements(By.TAG_NAME, 'td')
            teams.append(stats[0].text.strip())
            confs.append(stats[1].text)
            divs.append(stats[2].text)
            wins.append(stats[3].text.strip())
            losses.append(stats[4].text.strip())
            wlps.append(stats[5].text.strip())
            movs.append(stats[6].text.strip())
            orts.append(stats[7].text.strip())
            drts.append(stats[8].text.strip())
            nrts.append(stats[9].text.strip())
            movas.append(stats[10].text.strip())
            ortas.append(stats[11].text.strip())
            drtas.append(stats[12].text.strip())
            nrtas.append(stats[13].text.strip())
                
        driver.quit()
        
        # Create a DataFrame for the current year
        data = list(zip(rks, teams, confs, divs, wins, losses, wlps, movs, orts, drts, nrts, movas, ortas, drtas, nrtas))
        df_year = pd.DataFrame(data, columns=cols)
        df_year['Season'] = f"{year - 1}-{year}"  # Add a column for the year
        
        # Append the year data to the all_data list
        all_data.append(df_year)

    # Concatenate all the data frames
    final_df = pd.concat(all_data, axis=0, ignore_index=True)

    
    return final_df

In [ ]:
hp_df = get_historical_players()
hp_df

In [ ]:
hp_df.to_csv("../Data/historical_players.csv", index=False)

---
### Function to get Stats*

In [2]:
def get_stats(year_interval: str='2020:2022', div_id: str='totals_team'):
    all_data = []
    start_year, end_year = year_interval.split(':')
    for year in range(int(start_year), int(end_year) + 1):
        # Path to Edge WebDriver
        service = Service('C:\Program Files\WebDriver\msedgedriver.exe')

        # Initialize ME browser
        driver = webdriver.Edge(service=service)
        # Construct the URL for each year
        url = f"{BASE_URL}leagues/NBA_{year}.html"
        driver.get(url)
        
        try: 
            tbl = driver.find_element(By.ID, div_id)
        except Exception:
            driver.quit()
            continue
        
        # Extract column names
        header = tbl.find_element(By.TAG_NAME, 'thead')
        cols = header.text.strip().split(' ')

        rows = tbl.find_element(By.TAG_NAME, 'tbody')
        data_rows = rows.find_elements(By.TAG_NAME, 'tr')
        teams, Gs, MPs, FGs, FGAs, FGPs, thPs, thPAs, thPPs, tPs, tPAs, tPPs, FTs, FTAs, FTPs, ORBs, DRBs, TRBs, ASTs, STLs, BLKs, TOVs, PFs, PTSs  = [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []
        ranks = []
        # Extract data
        for row in data_rows:
            try:
                rank = row.find_element(By.TAG_NAME, 'th').text
                rank = int(rank)
            except:
                continue
            ranks.append(rank)
            
            stats = row.find_elements(By.TAG_NAME, 'td')
            teams.append(stats[0].text.replace('*',''))
            Gs.append(stats[1].text)
            MPs.append(stats[2].text)
            FGs.append(stats[3].text)
            FGAs.append(stats[4].text)
            FGPs.append(stats[5].text)
            thPs.append(stats[6].text)
            thPAs.append(stats[7].text)
            thPPs.append(stats[8].text)
            tPs.append(stats[9].text)
            tPAs.append(stats[10].text)
            tPPs.append(stats[11].text)
            FTs.append(stats[12].text)
            FTAs.append(stats[13].text)
            FTPs.append(stats[14].text)
            ORBs.append(stats[15].text)
            DRBs.append(stats[16].text)
            TRBs.append(stats[17].text)
            ASTs.append(stats[18].text)
            STLs.append(stats[19].text)
            BLKs.append(stats[20].text)
            TOVs.append(stats[21].text)
            PFs.append(stats[22].text)
            PTSs.append(stats[23].text)
            
        driver.quit()
        
        # Create a DataFrame for the current year
        data = list(zip(ranks, teams, Gs, MPs, FGs, FGAs, FGPs, thPs, thPAs, thPPs, tPs, tPAs, tPPs, FTs, FTAs, FTPs, ORBs, DRBs, TRBs, ASTs, STLs, BLKs, TOVs, PFs, PTSs))
        df_year = pd.DataFrame(data, columns=cols)
        df_year['Season'] = f"{year - 1}-{year}"  # Add a column for the year
        
        # Append the year data to the all_data list
        all_data.append(df_year)

    # Concatenate all the data frames
    final_df = pd.concat(all_data, axis=0, ignore_index=True)

    
    return final_df

### Total Stats

In [ ]:
totals_df = get_stats('1995:2024', 'totals-team')
totals_df

In [ ]:
totals_df.to_csv("../Data/total_stats.csv", header=True, index=False)

### Per Game Stats

In [ ]:
pgs_df = get_stats('1995:2024', 'per_game-team')
pgs_df

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Season
0,1,Orlando Magic,82,243.0,42.2,84.1,.502,6.4,17.2,.370,...,14.0,30.0,44.0,27.8,8.2,6.0,15.8,21.0,110.9,1994-1995
1,2,Phoenix Suns,82,241.8,40.9,85.0,.482,7.1,19.3,.369,...,12.5,29.3,41.8,26.8,8.4,3.8,14.2,22.4,110.6,1994-1995
2,3,Seattle SuperSonics,82,241.8,40.4,82.2,.491,6.0,15.9,.376,...,13.0,28.5,41.5,25.8,11.2,4.8,15.8,25.2,110.4,1994-1995
3,4,San Antonio Spurs,82,242.1,39.5,81.5,.484,5.3,14.1,.375,...,12.5,32.5,45.0,23.4,8.0,5.6,15.2,22.8,106.6,1994-1995
4,5,Utah Jazz,82,241.2,39.5,77.3,.512,3.7,9.8,.376,...,10.7,29.4,40.1,27.5,9.2,4.8,15.7,24.9,106.4,1994-1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883,26,Miami Heat,82,240.9,39.8,85.6,.465,12.5,33.7,.370,...,9.3,33.0,42.3,25.8,7.5,3.4,12.7,17.3,110.1,2023-2024
884,27,Detroit Pistons,82,240.9,40.9,88.2,.463,11.0,31.7,.348,...,10.5,32.8,43.3,25.5,6.5,4.7,15.2,20.6,109.9,2023-2024
885,28,Charlotte Hornets,82,240.6,40.0,87.0,.460,12.1,34.0,.355,...,9.3,31.0,40.3,24.8,6.9,4.5,13.8,18.0,106.6,2023-2024
886,29,Portland Trail Blazers,82,242.4,39.4,89.7,.439,11.5,33.2,.345,...,12.6,30.1,42.7,23.1,7.6,4.3,15.2,20.2,106.4,2023-2024


In [ ]:
pgs_df.to_csv("../Data/per_game_stats.csv", header=True, index=False)

### Per 100 Poss Stats

In [ ]:
per_poss_df = get_stats('1995:2024', 'per_poss-team')
per_poss_df

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Season
0,1,Orlando Magic,82,19930,43.8,87.4,.502,6.6,17.9,.370,...,14.5,31.1,45.7,28.9,8.5,6.2,16.4,21.9,115.1,1994-1995
1,2,Seattle SuperSonics,82,19830,42.0,85.5,.491,6.2,16.5,.376,...,13.5,29.6,43.2,26.8,11.6,5.0,16.4,26.2,114.8,1994-1995
2,3,Phoenix Suns,82,19830,42.3,87.9,.482,7.4,20.0,.369,...,13.0,30.3,43.3,27.7,8.7,3.9,14.7,23.2,114.5,1994-1995
3,4,Utah Jazz,82,19780,42.5,83.0,.512,3.9,10.5,.376,...,11.4,31.6,43.0,29.6,9.9,5.1,16.9,26.8,114.3,1994-1995
4,5,San Antonio Spurs,82,19855,41.3,85.4,.484,5.5,14.8,.375,...,13.1,34.0,47.1,24.5,8.4,5.8,15.9,23.9,111.7,1994-1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883,26,San Antonio Spurs,82,19830,41.2,89.0,.462,12.4,35.7,.347,...,10.2,33.3,43.4,29.3,7.0,6.2,14.9,16.9,110.0,2023-2024
884,27,Detroit Pistons,82,19755,40.8,88.1,.463,11.0,31.7,.348,...,10.5,32.8,43.3,25.4,6.5,4.7,15.2,20.6,109.7,2023-2024
885,28,Charlotte Hornets,82,19730,41.0,89.2,.460,12.4,34.9,.355,...,9.6,31.7,41.3,25.4,7.0,4.6,14.1,18.4,109.3,2023-2024
886,29,Portland Trail Blazers,82,19880,40.1,91.3,.439,11.7,33.8,.345,...,12.9,30.7,43.5,23.5,7.8,4.4,15.5,20.5,108.3,2023-2024


In [ ]:
per_poss_df.to_csv("../Data/per_100_poss_stats.csv", header=True, index=False)